Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

## Extractive Summarization on CNN/DM Dataset using Transformer Version of BertSum


### Summary

This notebook demonstrates how to fine tune Transformers for extractive text summarization. Utility functions and classes in the NLP Best Practices repo are used to facilitate data preprocessing, model training, model scoring, result postprocessing, and model evaluation.

BertSum refers to  [Fine-tune BERT for Extractive Summarization](https://arxiv.org/pdf/1903.10318.pdf) with [published example](https://github.com/nlpyang/BertSum/). And the Transformer version of Bertsum refers to our modification of BertSum and the source code can be accessed at (https://github.com/daden-ms/BertSum/). 

Extractive summarization are usually used in document summarization where each input document consists of mutiple sentences. The preprocessing of the input training data involves assigning label 0 or 1 to the document sentences based on the give summary. The summarization problem is also simplfied to classifying whether a document sentence should be included in the summary. 

The figure below illustrates how BERTSum can be fine tuned for extractive summarization task. [CLS] token is inserted at the beginning of each sentence, so is [SEP] token at the end. Interval segment embedding and positional embedding are added upon the token embedding as the input of the BERT model. The [CLS] token representation is used as sentence embedding and only the [CLS] tokens are used as the input for the summarization model. The summarization layer predicts the probability for each  sentence being included in the summary. Techniques like trigram blocking can be used to improve model accuarcy.   

<img src="https://nlpbp.blob.core.windows.net/images/BertSum.PNG">


### Before You Start

The running time shown in this notebook is on a Standard_NC24s_v3 Azure Ubuntu Virtual Machine with 4 NVIDIA Tesla V100 GPUs. 
> **Tip**: If you want to run through the notebook quickly, you can set the **`QUICK_RUN`** flag in the cell below to **`True`** to run the notebook on a small subset of the data and a smaller number of epochs. 

Using only 1 NVIDIA Tesla V100 GPUs, 16GB GPU memory configuration,
- for data preprocessing, it takes around 10 minutes to preprocess the data for quick run. Otherwise it takes ~2 hours to finish the data preprocessing. This time estimation assumes that the chosen transformer model is "distilbert-base-uncased" and the sentence selection method is "greedy", which is the default. The preprocessing time can be significantly longer if the sentence selection method is "combination", which can achieve better model performance.

- for model fine tuning, it takes around 30 minutes for quick run. Otherwise, it takes around ~3 hours to finish. This estimation assumes the chosen encoder method is "transformer". The model fine tuning time can be shorter if other encoder method is chosen, which may result in worse model performance. 


In [1]:
## Set QUICK_RUN = True to run the notebook on a small subset of data and a smaller number of epochs.
QUICK_RUN = True
## Set USE_PREPROCSSED_DATA = True to skip the data preprocessing
USE_PREPROCSSED_DATA = False

### Configuration


In [2]:
%load_ext autoreload

In [3]:
%autoreload 2

In [4]:
import os
import sys
from tempfile import TemporaryDirectory
import torch

nlp_path = os.path.abspath("../../")
if nlp_path not in sys.path:
    sys.path.insert(0, nlp_path)

from utils_nlp.common.pytorch_utils import get_device
from utils_nlp.dataset.cnndm import CNNDMBertSumProcessedData, CNNDMSummarizationDataset
from utils_nlp.eval.evaluate_summarization import get_rouge
from utils_nlp.models.transformers.extractive_summarization import (
    get_dataloader,
    ExtractiveSummarizer,
    ExtSumProcessedData,
    ExtSumProcessor,
)

[nltk_data] Downloading package punkt to /home/daden/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
I1224 02:52:40.350996 139807676634944 file_utils.py:40] PyTorch version 1.2.0 available.



### Configuration: choose the transformer model to be used

In [5]:
# Transformer model being used
MODEL_NAME = "distilbert-base-uncased"

Also, we need to install the dependencies for pyrouge.

# dependencies for ROUGE-1.5.5.pl
Run the following commands in your terminal to install XML parsing C library.

1. sudo apt-get update
1. sudo apt-get install expat
1. sudo apt-get install libexpat-dev -y

Run the following commands in your terminal to install other pre-requistes for using pyrouge.
1. sudo cpan install XML::Parser
1. sudo cpan install XML::Parser::PerlSAX
1. sudo cpan install XML::DOM

Download ROUGE-1.5.5 from https://github.com/andersjo/pyrouge/tree/master/tools/ROUGE-1.5.5.
Run the following command in your terminal.
* pyrouge_set_rouge_path $ABSOLUTE_DIRECTORY_TO_ROUGE-1.5.5.pl

### Data Preprossing

The dataset we used for this notebook is CNN/DM dataset which contains the documents and accompanying questions from the news articles of CNN and Daily mail. The highlights in each article are used as summary. The dataset consits of ~289K training examples, ~11K valiation examples and ~11K test examples.  You can choose the [Option 1] below preprocess the data or [Option 2] to use the preprocessed version at [BERTSum published example](https://github.com/nlpyang/BertSum/). You don't need to manually download any of these two data sets as the code below will handle downloading. Functions defined specific in [cnndm.py](../../utils_nlp/dataset/cnndm.py) are unique to CNN/DM dataset that's preprocessed by harvardnlp. However, it provides a skeleton of how to preprocessing text into the format that model preprocessor takes: sentence tokenization and work tokenization. 

##### Details of Data Preprocessing

The purpose of preprocessing is to process the input articles to the format that model finetuning needed. Assuming you have (1) all articles and (2) target summaries, each in a file and line-breaker seperated, the steps to preprocess the data are:
1. sentence tokenization
2. word tokenization
3. **label** the sentences in the article with 1 meaning the sentence is selected and 0 meaning the sentence is not selected. The algorithms for the sentence selection are "greedy" and "combination" and can be found in [sentence_selection.py](../../utils_nlp/dataset/sentence_selection.py)
3. convert each example to  the desired format for extractive summarization
    - filter the sentences in the example based on the min_src_ntokens argument. If the lefted total sentence number is less than min_nsents, the example is discarded.
    - truncate the sentences in the example if the length is greater than max_src_ntokens
    - truncate the sentences in the example and the labels if the total number of sentences is greater than max_nsents
    - [CLS] and [SEP] are inserted before and after each sentence
    - wordPiece tokenization or Byte Pair Encoding (BPE) subword tokenization
    - truncate the example to 512 tokens
    - convert the tokens into token indices corresponding to the transformer tokenizer's vocabulary.
    - segment ids are generated and added
    - [CLS] token positions are logged
    - [CLS] token labels are truncated if it's greater than 512, which is the maximum input length that can be taken by the transformer model.
    
    
Note that the original BERTSum paper use Stanford CoreNLP for data preprocessing, here we use NLTK for data preprocessing. 

##### [Option 1] Preprocess  data (Please skil this part if you choose to use preprocessed data)
The code in following cell will download the CNN/DM dataset listed at https://github.com/harvardnlp/sent-summary/.

In [6]:
# the data path used to save the downloaded data file
DATA_PATH =  "/tmp/tmpm2eh8iau" #TemporaryDirectory().name
# The number of lines at the head of data file used for preprocessing. -1 means all the lines.
TOP_N = -1
if QUICK_RUN:
    TOP_N = 10000

In [7]:
train_dataset, test_dataset = CNNDMSummarizationDataset(top_n=TOP_N, local_cache_path=DATA_PATH)

100%|██████████| 489k/489k [00:07<00:00, 69.5kKB/s] 
I1224 02:52:47.868990 139807676634944 utils.py:173] Opening tar file /tmp/tmpm2eh8iau/cnndm.tar.gz.


Preprocess the data and save the data to disk.

In [8]:
processor = ExtSumProcessor(model_name=MODEL_NAME)
ext_sum_train = processor.preprocess(train_dataset, train_dataset.get_target(), oracle_mode="greedy")
ext_sum_test = processor.preprocess(test_dataset, test_dataset.get_target(),oracle_mode="greedy")

I1224 02:52:59.080935 139807676634944 tokenization_utils.py:379] loading file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at ./26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084


{'max_nsents': 200, 'max_src_ntokens': 2000, 'min_nsents': 3, 'min_src_ntokens': 5, 'use_interval': True}


In [9]:
save_path = os.path.join(DATA_PATH, "processed")
train_files = ExtSumProcessedData.save_data(
    ext_sum_train, is_test=False, save_path=save_path, chunk_size=2000
)
test_files = ExtSumProcessedData.save_data(
    ext_sum_test, is_test=True, save_path=save_path, chunk_size=2000
)

In [10]:
train_files

['/tmp/tmpm2eh8iau/processed/0_train',
 '/tmp/tmpm2eh8iau/processed/1_train',
 '/tmp/tmpm2eh8iau/processed/2_train',
 '/tmp/tmpm2eh8iau/processed/3_train',
 '/tmp/tmpm2eh8iau/processed/4_train']

In [11]:
test_files

['/tmp/tmpm2eh8iau/processed/0_test',
 '/tmp/tmpm2eh8iau/processed/1_test',
 '/tmp/tmpm2eh8iau/processed/2_test',
 '/tmp/tmpm2eh8iau/processed/3_test',
 '/tmp/tmpm2eh8iau/processed/4_test']

In [12]:
train_dataset, test_dataset = ExtSumProcessedData().splits(root=save_path)

#### Inspect Data

In [13]:
import torch
bert_format_data = torch.load(train_files[0])
print(len(bert_format_data))
bert_format_data[0].keys()

2000


dict_keys(['src', 'labels', 'segs', 'clss', 'src_txt', 'tgt_txt'])

In [14]:
bert_format_data[0]['labels']

[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

##### [Option 2] Reuse Preprocessed  data from [BERTSUM Repo](https://github.com/nlpyang/BertSum)

In [15]:
# the data path used to downloaded the preprocessed data from BERTSUM Repo.
# if you have downloaded the dataset, change the code to use that path where the dataset is.
#PROCESSED_DATA_PATH = TemporaryDirectory().name
data_path = "./temp_data5/"
PROCESSED_DATA_PATH = data_path

In [16]:
if USE_PREPROCSSED_DATA:
    CNNDMBertSumProcessedData.download(local_path=PROCESSED_DATA_PATH)
    train_dataset, test_dataset = ExtSumProcessedData().splits(root=PROCESSED_DATA_PATH)
    

### Model training
To start model training, we need to create a instance of ExtractiveSummarizer.
#### Choose the transformer model.
Currently ExtractiveSummarizer support two models:
- distilbert-base-uncase, 
- bert-base-uncase

Potentionally, roberta-based model and xlnet can be supported but needs to be tested.
#### Choose the encoder algorithm.
There are four options:
- baseline: it used a smaller transformer model to replace the bert model and with transformer summarization layer
- classifier: it uses pretrained BERT and fine-tune BERT with **simple logistic classification** summarization layer
- transformer: it uses pretrained BERT and fine-tune BERT with **transformer** summarization layer
- RNN: it uses pretrained BERT and fine-tune BERT with **LSTM** summarization layer

In [17]:
# notebook parameters
# the cache data path during find tuning
CACHE_DIR = TemporaryDirectory().name

# batch size, unit is the number of tokens
BATCH_SIZE = 3000

# GPU used for training
NUM_GPUS = 2

# Encoder name. Options are: 1. baseline, classifier, transformer, rnn.
ENCODER = "transformer"

# Learning rate
LEARNING_RATE=2e-3

# How often the statistics reports show up in training, unit is step.
REPORT_EVERY=100
    
if QUICK_RUN:
    # total number of steps for training
    MAX_STEPS=1e4
    # number of steps for warm up
    WARMUP_STEPS=5e3
    
else:
    MAX_STEPS=5e4
    WARMUP_STEPS=5e3
 

In [18]:
summarizer = ExtractiveSummarizer(MODEL_NAME, ENCODER, CACHE_DIR)

I1224 03:01:19.243283 139807676634944 file_utils.py:319] https://s3.amazonaws.com/models.huggingface.co/bert/distilbert-base-uncased-config.json not found in cache or force_download set to True, downloading to /tmp/tmp7bncjhvm
100%|██████████| 492/492 [00:00<00:00, 531307.30B/s]
I1224 03:01:19.403964 139807676634944 file_utils.py:334] copying /tmp/tmp7bncjhvm to cache at /tmp/tmp0ru6f_af/a41e817d5c0743e29e86ff85edc8c257e61bc8d88e4271bb1b243b6e7614c633.1ccd1a11c9ff276830e114ea477ea2407100f4a3be7bdc45d37be9e37fa71c7e
I1224 03:01:19.404998 139807676634944 file_utils.py:338] creating metadata file for /tmp/tmp0ru6f_af/a41e817d5c0743e29e86ff85edc8c257e61bc8d88e4271bb1b243b6e7614c633.1ccd1a11c9ff276830e114ea477ea2407100f4a3be7bdc45d37be9e37fa71c7e
I1224 03:01:19.406319 139807676634944 file_utils.py:347] removing temp file /tmp/tmp7bncjhvm
I1224 03:01:19.407042 139807676634944 configuration_utils.py:157] loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/distilbert

In [19]:
# batch_size is the number of tokens in a batch
train_dataloader = get_dataloader(train_dataset.get_stream(), is_labeled=True, batch_size=3000)

In [ ]:
summarizer.fit(
            train_dataloader,
            num_gpus=NUM_GPUS,
            gradient_accumulation_steps=2,
            max_steps=MAX_STEPS,
            lr=LEARNING_RATE,
            warmup_steps=WARMUP_STEPS,
            verbose=True,
            report_every=REPORT_EVERY,
            clip_grad_norm=False,
        )

/dadendev/anaconda3/envs/cm3/lib/python3.6/site-packages/torch/nn/parallel/_functions.py:61: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


loss: 11.160705, time: 42.738963, number of examples in current step: 5, step 100 out of total 10000
loss: 10.624718, time: 33.611201, number of examples in current step: 5, step 200 out of total 10000
loss: 10.389675, time: 32.889951, number of examples in current step: 5, step 300 out of total 10000
loss: 10.167939, time: 33.669455, number of examples in current step: 5, step 400 out of total 10000
loss: 10.233984, time: 32.727784, number of examples in current step: 5, step 500 out of total 10000
loss: 10.185300, time: 33.506980, number of examples in current step: 5, step 600 out of total 10000
loss: 10.074746, time: 33.141054, number of examples in current step: 5, step 700 out of total 10000
loss: 9.924833, time: 33.621255, number of examples in current step: 5, step 800 out of total 10000
loss: 9.977459, time: 32.910738, number of examples in current step: 5, step 900 out of total 10000
loss: 10.138617, time: 33.722555, number of examples in current step: 5, step 1000 out of tot

loss: 1.903633, time: 33.575222, number of examples in current step: 5, step 8300 out of total 10000
loss: 1.782045, time: 33.406267, number of examples in current step: 5, step 8400 out of total 10000
loss: 1.894894, time: 32.923071, number of examples in current step: 5, step 8500 out of total 10000
loss: 1.757045, time: 33.201561, number of examples in current step: 5, step 8600 out of total 10000
loss: 1.868286, time: 32.830063, number of examples in current step: 5, step 8700 out of total 10000
loss: 1.843515, time: 33.185594, number of examples in current step: 5, step 8800 out of total 10000
loss: 1.560396, time: 32.747909, number of examples in current step: 5, step 8900 out of total 10000


In [ ]:
summarizer.save_model("extsum_modelname_{0}_usepreprocess{1}_steps_{2}.pt".format(MODEL_NAME, USE_PREPROCSSED_DATA, MAX_STEPS))

In [ ]:
# for loading a previous saved model
#import torch
#summarizer.model = torch.load("cnndm_transformersum_distilbert-base-uncased_bertsum_processed_data.pt")

### Model Evaluation

[ROUGE](https://en.wikipedia.org/wiki/ROUGE_(metric)), or Recall-Oriented Understudy for Gisting Evaluation has been commonly used for evaluation text summarization.

In [ ]:
target = [i['tgt_txt'] for i in test_dataset]

In [ ]:
len(target)

In [ ]:
test_dataset[0].keys()

In [ ]:
%%time
prediction = summarizer.predict(test_dataset, num_gpus=NUM_GPUS, batch_size=128)

In [ ]:
len(prediction)

In [ ]:
rouge_transformer = get_rouge(prediction, target, "./results/")

In [ ]:
test_dataset[0]['tgt_txt']

In [ ]:
prediction[0]

In [ ]:
test_dataset[0]['src_txt']